# HW1
Name: Klaas Schuijtemaker

Student nr.: 11163119

Course: Information Retrieval

Date: 18 jan. 2017

## Step 1: Simulate Rankings of Relevance for E and P

First, we define several classes that will be used throughout the assignment.

In [1]:
# Document 
class Doc:
    id = 0     # document id
    rel = 'N'  # document relevance {N,R,HR}
    
    def __init__(self, id, rel):
        self.id = id
        self.rel = rel
        
    def __repr__(self):
        return 'Doc(' + str(self.id) + ',' + str(self.rel) + ')'
    
    def __eq__(self, doc2):
        return self.id == doc2.id
    
    def relevance(self):
        if self.rel == 'N':
            return 0
        elif self.rel == 'R':
            return 1
        elif self.rel == 'HR':
            return 5
        else:
            raise LookupError

In [2]:
# Test Doc -class
d = Doc(0,'HR')
print(d)
print('Document relevance:', d.relevance())

Doc(0,HR)
Document relevance: 5


In [3]:
# Query session: a list of documents
class QuerySession:
    doc_list = []   # list with Docs ordered by rank
    click_list = [] # list containigh the number of clicks per document
    
    def __init__(self, doc_list):
        self.doc_list = doc_list
        
    def __repr__(self):
        return 'QuerySession(' + str(self.doc_list) + ')'

In [4]:
# Test QuerySession -class
qs = QuerySession([Doc(0,'R'),Doc(1,'R'),Doc(2,'R'),Doc(3,'N'),Doc(4,'R')])
print(qs)
print('First doc:', qs.doc_list[0])

QuerySession([Doc(0,R), Doc(1,R), Doc(2,R), Doc(3,N), Doc(4,R)])
First doc: Doc(0,R)


In [5]:
# Pair: two QuerySessions.
class Pair:
    P = None # Production QuerySession
    E = None # Experimental QuerySession
    
    ap_measure = 0  # Average Precision -𝛥measure
    dcg_measure = 0 # Discounted Cumulative Gain -𝛥measure
    rbp_measure = 0 # Rank Biased Precision -𝛥measure
    err_measure = 0 # Expected Reciprocal Rank -𝛥measure
    
    td_interleaving = None # Team-Draft Interleaving
    p_interleaving = None  # Probabilistic Interleaving
    
    click_prop_td_rcm = 0  # The proportion of clicks on E with Team-Draft Interleaving and Random Click Model
    click_prop_td_sdcm = 0 # The proportion of clicks on E with Team-Draft Interleaving and Simplified Dependent Click model
    click_prop_p_rcm = 0   # The proportion of clicks on E with Probabilistic Interleaving and Random Click Model
    click_prop_p_sdcm = 0  # The proportion of clicks on E with Probabilistic Interleaving and Simplified Dependent Click model

    def __init__(self, P, E):
        self.P = P
        self.E = E
        
    def __repr__(self):
        return 'Pair(P:' + str(self.P) + ',\n     E:' + str(self.E) + ')'

In [6]:
# Test Pair -class
P = QuerySession([Doc(0,'R'),Doc(1,'N'),Doc(2,'R'),Doc(3,'N')])
E = QuerySession([Doc(4,'R'),Doc(5,'R'),Doc(6,'R'),Doc(7,'N')])
pair = Pair(P, E)
print(pair)
print('Production:', pair.P)

Pair(P:QuerySession([Doc(0,R), Doc(1,N), Doc(2,R), Doc(3,N)]),
     E:QuerySession([Doc(4,R), Doc(5,R), Doc(6,R), Doc(7,N)]))
Production: QuerySession([Doc(0,R), Doc(1,N), Doc(2,R), Doc(3,N)])


Here, we generate all pairs of rankings of relevance, for both the production P and experimental E.

In [7]:
# Generate all QuerySessions
graded_relevances = ['N','R','HR']
qs_array = []
doc_id = 0

for i1 in graded_relevances:
    for i2 in graded_relevances:
        for i3 in graded_relevances:
            for i4 in graded_relevances:
                for i5 in graded_relevances:
                    doc_list = [Doc(doc_id,i1), Doc(doc_id+1,i2), Doc(doc_id+2,i3), Doc(doc_id+3,i4), Doc(doc_id+4,i5)]
                    qs_array += [QuerySession(doc_list)]
                    doc_id += 5

# Add all possible combinations for P with E
pairs = []
for i1 in range(len(qs_array)):
    for i2 in range(len(qs_array)):
        pairs += [Pair(qs_array[i1], qs_array[i2])] 

In [8]:
# Check some pairs
print('Number of pairs:', len(pairs))
print(pairs[0])
print(pairs[1])

Number of pairs: 59049
Pair(P:QuerySession([Doc(0,N), Doc(1,N), Doc(2,N), Doc(3,N), Doc(4,N)]),
     E:QuerySession([Doc(0,N), Doc(1,N), Doc(2,N), Doc(3,N), Doc(4,N)]))
Pair(P:QuerySession([Doc(0,N), Doc(1,N), Doc(2,N), Doc(3,N), Doc(4,N)]),
     E:QuerySession([Doc(5,N), Doc(6,N), Doc(7,N), Doc(8,N), Doc(9,R)]))


## Step 2: Implement Evaluation Measures

In [9]:
import math

# Average Precision
#   Input:
#     doc_list: [Doc1, Doc2, ...]
#   Output: Average Precision
def ap_evaluation(doc_list):
    total_rel = 0
    out = 0
    for i in range(len(doc_list)):
        total_rel += 0 if doc_list[i].relevance() == 0 else 1
        out += total_rel / (i + 1)
    return 0 if total_rel == 0 else out / len(doc_list)

# Discounted Cumulative Gain
#   Input:
#     doc_list: [Doc1, Doc2, ...]
#     k: rank position
#   Output: DCG at rank k
def dcg_evaluation(doc_list, k=5):
    out = 0
    for i in range(1, k + 1):
        out += (2**doc_list[i - 1].relevance() - 1) / math.log(i + 1, 2)
    return out

# Rank Biased Precision
#   Input: 
#     doc_list: [Doc1, Doc2, ...]
#     p: persistence parameter
#   Output: RBP
def rbp_evaluation(doc_list, p=0.8):
    out = 0
    for i in range(1, len(doc_list) + 1):
        out += doc_list[i - 1].relevance() * p**i
    out *= 1 - p
    return out

# Expected Reciprocal Rank
#   Input: 
#     doc_list: [Doc1, Doc2, ...]
#     gmax: maximum relevance
#   Output: ERR
def err_evaluation(doc_list, gmax=4):
    out = 0
    p = 1
    for r in range(1, len(doc_list) + 1):
        ri = 2**(doc_list[r - 1].relevance() - 1) / 2**gmax
        out += p * ri / r
        p *= 1 - ri
    return out

In [11]:
# Test evaluations:
qs = QuerySession([Doc(0,'R'),Doc(1,'R'),Doc(2,'R'),Doc(3,'N'),Doc(4,'R')])
print('AP evaluation:', ap_evaluation(qs.doc_list))
print('DCG evaluation:', dcg_evaluation(qs.doc_list))
print('RBP evaluation:', rbp_evaluation(qs.doc_list))
print('ERR evaluation:', err_evaluation(qs.doc_list))

AP evaluation: 0.9099999999999999
DCG evaluation: 2.517782560805999
RBP evaluation: 0.45593599999999995
ERR evaluation: 0.12652254104614258


## Step 3: Calculate the 𝛥measure

In [12]:
# Calculate 𝛥measure = measure_e - measure_p
for pair in pairs:
    pair.ap_measure = ap_evaluation(pair.E.doc_list) - ap_evaluation(pair.P.doc_list)
    pair.dcg_measure = dcg_evaluation(pair.E.doc_list) - dcg_evaluation(pair.P.doc_list)
    pair.rbp_measure = rbp_evaluation(pair.E.doc_list) - rbp_evaluation(pair.P.doc_list)
    pair.err_measure = err_evaluation(pair.E.doc_list) - err_evaluation(pair.P.doc_list)

In [13]:
# Check 𝛥measure
print('Second pair:')
print(pairs[1])
print('𝛥measure = Average_Precision(E) - Average_Precision(P):', pairs[1].ap_measure)

Second pair:
Pair(P:QuerySession([Doc(0,N), Doc(1,N), Doc(2,N), Doc(3,N), Doc(4,N)]),
     E:QuerySession([Doc(5,N), Doc(6,N), Doc(7,N), Doc(8,N), Doc(9,R)]))
𝛥measure = Average_Precision(E) - Average_Precision(P): 0.04


## Step 4: Implement Interleaving

In [14]:
import random

# Team-Draft Interleaving
#   Input:  pair
#   Output: {'interleaved':i_list, 'team_a':team_a, 'team_b':team_b}
def td_interleaving(pair):
    i1 = 0
    i2 = 0
    team_a = []
    team_b = []
    i_list = []
    while i1 < len(pair.P.doc_list) or i2 < len(pair.E.doc_list):
        if i1 < i2 or (i1 == i2 and random.getrandbits(1) == 1):
            i_list += [pair.P.doc_list[i1]]
            team_a += [pair.P.doc_list[i1]]
            i1 += 1
        else:
            i_list += [pair.E.doc_list[i2]]
            team_b += [pair.E.doc_list[i2]]
            i2 += 1

        while i1 < len(pair.P.doc_list) and pair.P.doc_list[i1] in i_list:
            i1 += 1
        while i2 < len(pair.E.doc_list) and pair.E.doc_list[i2] in i_list:
            i2 += 1
    return {'interleaved': QuerySession(i_list), 'team_a': team_a, 'team_b': team_b}

In [15]:
import random

# Probabilistic Interleaving
#   Input:  pair
#   Output: {'interleaved':i_list, 'team_a':team_a, 'team_b':team_b}
def p_interleaving(pair, t=3):
    # TODO: ...
    return 0

In [16]:
# Test Team-Draft Interleaving:
P = QuerySession([Doc(1,'R'),Doc(2,'R'),Doc(3,'N'),Doc(4,'R')])
E = QuerySession([Doc(5,'R'),Doc(3,'N'),Doc(6,'R'),Doc(7,'R')])
pair = Pair(P, E)

td = td_interleaving(pair)
print('Test with Team-Draft Interleaving:')
print('Interleaved:', td['interleaved'])
print('Team A:', td['team_a'])
print('Team B:', td['team_b'])

Test with Team-Draft Interleaving:
Interleaved: QuerySession([Doc(1,R), Doc(5,R), Doc(3,N), Doc(2,R), Doc(6,R), Doc(4,R), Doc(7,R)])
Team A: [Doc(1,R), Doc(2,R), Doc(4,R)]
Team B: [Doc(5,R), Doc(3,N), Doc(6,R), Doc(7,R)]


In [17]:
# Do interleaving on all pairs
for pair in pairs:
    pair.td_interleaving = td_interleaving(pair)
    pair.p_interleaving = p_interleaving(pair)

In [18]:
# Check Team-Draft interleaving on a pair
pairs[50].td_interleaving

{'interleaved': QuerySession([Doc(250,N), Doc(0,N), Doc(251,R), Doc(1,N), Doc(252,HR), Doc(2,N), Doc(3,N), Doc(253,R), Doc(4,N), Doc(254,HR)]),
 'team_a': [Doc(0,N), Doc(1,N), Doc(2,N), Doc(3,N), Doc(4,N)],
 'team_b': [Doc(250,N), Doc(251,R), Doc(252,HR), Doc(253,R), Doc(254,HR)]}

## Step 5: Implement User Clicks Simulation

In [19]:
# Add support for nested dictonaries: {key1: {key2: value2}}
class NestedDict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

# Load a Yandex Click Log File:
#   Input:
#     file: Yandex Click Log File
#   Output: session_map[session_id][query_id] containing alls QuerySession in the Yandex file
def load_yandex_click_log(file):
    session_map = NestedDict()
    qs = None
    for line in file:
        cells = [cell.strip() for cell in line.split('\t')]
        session_id = int(cells[0])
            
        if cells[2] == 'Q':
            query_id = int(cells[3])
            if query_id not in session_map[session_id]:
                list_of_urls = [int(cell) for cell in cells[5:]]
                doc_list = [Doc(url_id,'None') for url_id in list_of_urls]
                qs = QuerySession(doc_list)
                qs.click_list = [0] * len(qs.doc_list)
                session_map[session_id][query_id] = qs
        
        if cells[2] == 'C':
            url_id = int(cells[3])
            for i,doc in enumerate(qs.doc_list):
                if doc.id == url_id:
                    qs.click_list[i] += 1
                    break;
    return session_map

In [20]:
# Load file into map
f = open('YandexRelPredChallenge.txt', 'r')
session_map = load_yandex_click_log(f)
f.close()

In [21]:
# Check a QuerySession
qs = session_map[0][1974];
print('Details of QuerySession with session_id: 0 and query_id: 1974')
print('Documents:', doc_list)
print('Clicks per Doc:', qs.click_list)

Details of QuerySession with session_id: 0 and query_id: 1974
Documents: [Doc(1210,HR), Doc(1211,HR), Doc(1212,HR), Doc(1213,HR), Doc(1214,HR)]
Clicks per Doc: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]


In [22]:
# Random click model estimate the probability of a click
#   Input:
#     session_map[session_id][query_id]
#   Output: Rho, the probability of a click
def rcm_get_click_prob(session_map):
    click_count = 0
    doc_count = 0
    for query_map in session_map.values():
        for session_query in query_map.values():
            for clicks in session_query.click_list:
                click_count += clicks                  # Count the number of clicks
            doc_count += len(session_query.doc_list)   # Count the number of documents
    return click_count / doc_count                     # Rho = click_count / doc_count

In [23]:
# Random click model, estimate whether a document is clicked
#   Input:
#     rcm_click_prob: the probability of a click
#   Output: True if this document was clicked, False otherwise
def bernoulli(click_prob):
    return random.random() < click_prob

In [24]:
# Simplified Dependent Click Model estimate the continuation probability
#   Input:
#     session_map[session_id][query_id]
#   Output: Lambda r, a list[rank] with the continuation probability at each rank
def sdcm_get_continuation_probs(session_map):
    click_list = [0] * 10
    click_not_last_list = [0] * 10
    for query_map in session_map.values():
        for session_query in query_map.values():
            click_ranks = [r for r, click in enumerate(session_query.click_list) if click]
            if len(click_ranks) > 0:
                click_list[click_ranks[-1]] += session_query.click_list[click_ranks[-1]]
                for rank in click_ranks[:-1]:
                    click_list[rank] += session_query.click_list[rank]
                    click_not_last_list[rank] += session_query.click_list[rank]
    return [a / b for a, b in zip(click_not_last_list, click_list)]

In [25]:
# Simplified Dependent Click model estimate the attraction probability
#   Input:
#     query_session
#   Output: Alpha r, a list[rank] with the attraction probability at each rank
def sdcm_get_attraction_probs(query_session):
    return [d.relevance() / 4 for d in query_session.doc_list]

In [26]:
# Simplified Dependent Click model, estimate whether a document is clicked
#   Input:
#     query_session: a query session containing a ranked list of documents
#     cont_probs: Lambda r, a list[rank] with the continuation probability at each rank
#     attr_probs: Alpha r, a list[rank] with the attraction probability at each rank
#   Output: A click list[rank]: [True, False, False]
def sdcm_was_clicked(query_session, cont_probs, attr_probs):
    exam = 1
    click_list = []
    for rank in range(len(query_session.doc_list)):
        cont = cont_probs[rank]
        attr = attr_probs[rank]
        if exam == 0:
            click = False
            exam = 0
        else:
            click = bernoulli(attr)
            exam = bernoulli(1 - click + cont * click)
        click_list.append(click)
    return click_list

In [27]:
# calculate probabilities
rcm_click_prob = rcm_get_click_prob(session_map)
sdcm_cont_probs = sdcm_get_continuation_probs(session_map)

In [28]:
# Check probabilities
print('Random click probability:', round(rcm_click_prob, 2))
print('Continuation probabilities:', [round(cp, 2) for cp in sdcm_cont_probs])
print('\n')

# test-QuerySession
qs = QuerySession([Doc(0,'R'),Doc(1,'R'),Doc(2,'N'),Doc(3,'R'),Doc(4,'N')])
print('QuerySession for testing:')
print(qs)
print('\n')

# Test Random Click Model
rdm_click_list = [bernoulli(rcm_click_prob) for i in range(len(qs.doc_list))]
print('Random Click Model:')
print('Clicked-list of test QuerySession:', rdm_click_list)
print('\n')

# Test Simplified Dependent Click Model
sdcm_attr_probs = sdcm_get_attraction_probs(qs)
sdcm_click_list = sdcm_was_clicked(qs, sdcm_cont_probs, sdcm_attr_probs)
print('Simplified Dependent Click model:')
print('Attraction probabilities of test-QuerySession:', sdcm_attr_probs)
print('Clicked-list of test-QuerySession:', sdcm_click_list)

Random click probability: 0.14
Continuation probabilities: [0.37, 0.56, 0.59, 0.58, 0.57, 0.57, 0.54, 0.47, 0.4, 0.0]


QuerySession for testing:
QuerySession([Doc(0,R), Doc(1,R), Doc(2,N), Doc(3,R), Doc(4,N)])


Random Click Model:
Clicked-list of test QuerySession: [True, False, False, False, False]


Simplified Dependent Click model:
Attraction probabilities of test-QuerySession: [0.25, 0.25, 0.0, 0.25, 0.0]
Clicked-list of test-QuerySession: [False, False, False, False, False]


## Step 6: Simulate Interleaving Experiment

In [29]:
# Run Random Click Model on interleaved list
#   Input:
#     interleaf: {'interleaved':i_list, 'team_a':team_a, 'team_b':team_b}
#     n: number of runs
#   Output: proportion p of clicks for E
def run_rcm(interleaf, n=1000):
    interleaved = interleaf['interleaved']
    team_a = interleaf['team_a']
    team_b = interleaf['team_b']
    
    p_click_count = 0
    e_click_count = 0
    for i in range(n):
        for doc in interleaved.doc_list:
            if bernoulli(rcm_click_prob):
                if doc in team_a:
                    p_click_count += 1
                if doc in team_b:
                    e_click_count += 1

    total_count = p_click_count + e_click_count
    return 0 if total_count == 0 else e_click_count / total_count

In [30]:
# Run Simplified Dependend Click Model on interleaved list
#   Input:
#     interleaf: {'interleaved':i_list, 'team_a':team_a, 'team_b':team_b}
#     n: number of runs
#   Output: proportion p of clicks for E
def run_sdcm(interleaf, n=1000):
    interleaved = interleaf['interleaved']
    team_a = interleaf['team_a']
    team_b = interleaf['team_b']
    
    p_click_count = 0
    e_click_count = 0
    for i in range(n):
        sdcm_attr_probs = sdcm_get_attraction_probs(interleaved)
        click_list = sdcm_was_clicked(interleaved, sdcm_cont_probs, sdcm_attr_probs)
        for rank, click in enumerate(click_list):
            if click:
                doc = interleaved.doc_list[rank]
                if doc in team_a:
                    p_click_count += 1
                if doc in team_b:
                    e_click_count += 1

    total_count = p_click_count + e_click_count
    return 0 if total_count == 0 else e_click_count / total_count

In [31]:
# Run experiment

for pair in pairs:
    pair.click_prop_td_rcm = run_rcm(pair.td_interleaving)
    pair.click_prop_td_sdcm = run_sdcm(pair.td_interleaving)
    #pair.click_prop_p_rcm = run_rcm(pair.p_interleaving)
    #pair.click_prop_p_sdcm = run_sdcm(pair.p_interleaving)

## Step 7: Results and Analysis

In [32]:
# Get Pearson correlation coefficient between measures and click-proportions

from scipy.stats.stats import pearsonr

ap_measures = []
dcg_measures = []
rbp_measures = []
err_measures = []
click_prop_td_rcms = []
click_prop_td_sdcms = []

for pair in pairs:
    ap_measures += [pair.ap_measure]
    dcg_measures += [pair.dcg_measure]
    rbp_measures += [pair.rbp_measure]
    err_measures += [pair.err_measure]
    click_prop_td_rcms += [pair.click_prop_td_rcm]
    click_prop_td_sdcms += [pair.click_prop_td_sdcm]

pcc_ap_rcm = pearsonr(ap_measures, click_prop_td_rcms)
pcc_dcg_rcm = pearsonr(dcg_measures, click_prop_td_rcms)
pcc_rbp_rcm = pearsonr(rbp_measures, click_prop_td_rcms)
pcc_err_rcm = pearsonr(err_measures, click_prop_td_rcms)
pcc_ap_sdcm = pearsonr(ap_measures, click_prop_td_sdcms)
pcc_dcg_sdcm = pearsonr(dcg_measures, click_prop_td_sdcms)
pcc_rbp_sdcm = pearsonr(rbp_measures, click_prop_td_sdcms)
pcc_err_sdcm = pearsonr(err_measures, click_prop_td_sdcms)


### Results

In [33]:
print('Pearson correlation coefficient between measures and click-proportions using Team-Draft Interleaving:\n')
print('Average Precision (AP)           and Random Click Model (RCM):               ', pcc_ap_rcm[0])
print('Expected Reciprocal Rank (ERR)   and Random Click Model (RCM):               ', pcc_err_rcm[0])
print('Discounted Cumulative Gain (DCG) and Random Click Model (RCM):               ', pcc_dcg_rcm[0])
print('Rank Biased Precision (RBP)      and Random Click Model (RCM):               ', pcc_rbp_rcm[0])
print('Average Precision (AP)           and Simplified Dependent Click model (SDCM):', pcc_ap_sdcm[0])
print('Expected Reciprocal Rank (ERR)   and Simplified Dependent Click model (SDCM):', pcc_err_sdcm[0])
print('Discounted Cumulative Gain (DCG) and Simplified Dependent Click model (SDCM):', pcc_dcg_sdcm[0])
print('Rank Biased Precision (RBP)      and Simplified Dependent Click model (SDCM):', pcc_rbp_sdcm[0])

Pearson correlation coefficient between measures and click-proportions using Team-Draft Interleaving:

Average Precision (AP)           and Random Click Model (RCM):                -0.00127711369941
Expected Reciprocal Rank (ERR)   and Random Click Model (RCM):                -0.00035283570723
Discounted Cumulative Gain (DCG) and Random Click Model (RCM):                0.00125283413329
Rank Biased Precision (RBP)      and Random Click Model (RCM):                0.000947231375213
Average Precision (AP)           and Simplified Dependent Click model (SDCM): 0.616618771709
Expected Reciprocal Rank (ERR)   and Simplified Dependent Click model (SDCM): 0.834751718387
Discounted Cumulative Gain (DCG) and Simplified Dependent Click model (SDCM): 0.848015207088
Rank Biased Precision (RBP)      and Simplified Dependent Click model (SDCM): 0.856530398587


### Analysis
In this research we simulated two non-existing information retrieval algorithms. The algorithm in production, called P, and an experimental algorithm, called E. Team-draft Interleaving was used to setup an experiment between E and P. In this experiment, we tested four evaluation methods and two click models.

The tested click models:
- Random Click Model (RCM)
- Simplified Dependent Click Model (SDCM)

The tested evaluation metods:
- Average Precision (AP)
- Discounted Cumulative Gain (DCG)
- Rank Biased Precision (RBP)
- Expected Reciprocal Rank (ERR)

Algorithm E and P where evaluated using all evaluation methods. By taking the difference of the evaluations, we calculated the measure. The measure is expected to be positive when E has more relavant document on top of the ranking list then P.
Then, every click model was tested on the interleaved ranking. This resulted in a proportion of clicks for E. The proportion of clicks for E is expected to be 'high' when E has more relavant document on top of the ranking list then P. 

Both the measure and the proportion of clicks are expected to be 'high' when E has more relevant documents onn top of the ranking list, then P. This means that the measure and the proportion of click are correlated. When a 'good' click model is evaluated with a 'good' evaluation method, the correlation will approach 1. When a either the click model is 'bad' or when the evaluation method is 'bad', the correlation will approach 0. 

The RCM does a bad job at simulating user clicks. This can be seen in the results. The correlation coefficient of RCM approaches 0 at all of the evaluation methods.


The SDCM does a better job at simulating user clicks. The correlation coefficient of SDCM is much closer to 1.

If we assume that the SDCM is 'good' at simulating user clicks, we can also evaluate the evaluation methods. AP is bad at evaluating the SDCM. Since the correlation between SDCM and AP is lower then the correlation between SDCM and the other evaluation methods.

Finally, SDCM and RBP have the highest correlation. This sugests SDCM is the best click model and that RBP is the best evaluation method. However, the significance of this result, has not been measured. And since the correlation between SDCM and ERR,DCG,RBP, are all more or less similar, the result should be taken with a grain of salt.